In [1]:
import jpeg_ls
import imageio as iio

# img is in RGBA format , opacity 0 for png images
img = iio.imread("./images/machine.png", format='png')

# get grayscale values for each color
red_img = [[pixel[0] for pixel in row] for row in img]
green_img = [[pixel[1] for pixel in row] for row in img]
blue_img = [[pixel[2] for pixel in row] for row in img]

M = 256     # alphabet set

res_seq_red = jpeg_ls.de_correlation(red_img, M)
res_seq_green = jpeg_ls.de_correlation(green_img, M)
res_seq_blue = jpeg_ls.de_correlation(blue_img, M)

encode_red = jpeg_ls.encode_residual_seq(res_seq_red, M)
encode_green = jpeg_ls.encode_residual_seq(res_seq_green, M)
encode_blue = jpeg_ls.encode_residual_seq(res_seq_blue, M)

newfile = open("./images/compressed_machine", "wb")
newfile.write(encode_red.tobytes())
newfile.write(encode_green.tobytes())
newfile.write(encode_blue.tobytes())

newfile.close()



11000


D:\Course Projects\ELL786\ELL786_A2\jpeg_ls.py:72: RuntimeWarning: overflow encountered in ubyte_scalars
  g3 = c-a
D:\Course Projects\ELL786\ELL786_A2\jpeg_ls.py:93: RuntimeWarning: overflow encountered in ubyte_scalars
  cum_error[context_val] += grayscale_img[i][j] - predicted_seq[i][j]
D:\Course Projects\ELL786\ELL786_A2\jpeg_ls.py:70: RuntimeWarning: overflow encountered in ubyte_scalars
  g1 = d-b
D:\Course Projects\ELL786\ELL786_A2\jpeg_ls.py:71: RuntimeWarning: overflow encountered in ubyte_scalars
  g2 = b-c
D:\Course Projects\ELL786\ELL786_A2\jpeg_ls.py:65: RuntimeWarning: overflow encountered in ubyte_scalars
  predicted_seq[i][j] = min(a, b) if c >= max(a, b) else max(a, b) if c <= min(a, b) else a+b-c


AttributeError: 'list' object has no attribute 'tobytes'